In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

# from vivarium_research_prl.noise import corruption, fake_names, noisify
# from vivarium_research_prl.find_kids import datasets, noisify_data
# import vivarium_research_prl.find_kids as find_kids

from splink.duckdb.duckdb_linker import DuckDBLinker
import splink.duckdb.duckdb_comparison_library as cl

# For viewing waterfall charts and precision-recall curve
import altair as alt
alt.renderers.enable('mimetype')
alt.renderers.enable('html')

# For viewing the comparison viewer dashboard
from IPython.display import IFrame

!date
!whoami
!uname -a
!pwd

Thu 19 Jan 2023 02:06:11 PM PST
ndbs
Linux int-slurm-sarchive-p0012 5.4.0-88-generic #99-Ubuntu SMP Thu Sep 23 17:29:00 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/linkage/wic_case_study


In [2]:
%load_ext autoreload
%autoreload 2

# Read in saved data

In [3]:
!ls -l data

total 5484
-rw-rw-r-- 1 ndbs IHME-users 5535621 Jan 11 15:47 prepared_2020_census_20221014.csv
-rw-rw-r-- 1 ndbs IHME-users   66364 Jan 11 15:47 prepared_wic_20221014.csv


In [4]:
data_dir = 'data'
dtypes = {'zipcode': str} # Make sure zipcodes are str not int
df_census = pd.read_csv(
    f'{data_dir}/prepared_2020_census_20221014.csv',
    dtype=dtypes,
    index_col=0,
)
df_wic = pd.read_csv(
    f'{data_dir}/prepared_wic_20221014.csv',
    dtype=dtypes,
    index_col=0,
)
print(df_census.shape, df_wic.shape, sep='\n')

(47529, 12)
(633, 12)


In [5]:
df_census.dtypes

first_name                     object
middle                         object
last_name                      object
date_of_birth                  object
age                           float64
sex                            object
race_ethnicity                 object
relation_to_household_head     object
address                        object
zipcode                        object
unique_id                       int64
household_id                  float64
dtype: object

In [6]:
df_wic.dtypes

first_name                     object
middle                         object
last_name                      object
date_of_birth                  object
sex                            object
race_ethnicity                 object
address                        object
zipcode                        object
household_id                    int64
unique_id                       int64
age                           float64
relation_to_household_head    float64
dtype: object

In [7]:
df_census

,first_name,middle,last_name,date_of_birth,age,sex,race_ethnicity,relation_to_household_head,address,zipcode,unique_id,household_id
0,Margaret,J,Clark,1951-07-27,68.0,Female,Black,Reference person,"1344 winoka rd brooksville, fl",34601,1,NaN
1,Jeffrey,V,Littlejohn,1967-05-03,52.0,Male,Black,Reference person,"927 23rd st clearwater, fl",34698,2,NaN
2,Briana,A,Jackson,2006-09-07,13.0,Female,Black,Biological child,"927 23rd st clearwater, fl",34698,3,NaN
3,Benjamin,D,Cox,1998-10-21,21.0,Male,Black,Stepchild,"927 23rd st clearwater, fl",34698,4,NaN
4,Willie,NaN,Tucker,1947-10-09,72.0,Male,White,Reference person,"8904 167th place fleming island, fl",32003,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
49994,Marcus,S,Roman,1988-07-08,31.0,Male,Multiracial or Other,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",33021,47525,NaN
49996,Nathaniel,J,Campbell,1941-01-08,79.0,Male,White,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",33021,47526,NaN
49997,Christian,C,Rosales,1983-12-16,36.0,Male,Latino,Institutionalized GQ pop,"701 haber rd vero beach, fl",32968,47527,NaN
49998,Phillip,J,Morton,1985-06-11,34.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",33919,47528,NaN


In [8]:
df_wic

,first_name,middle,last_name,date_of_birth,sex,race_ethnicity,address,zipcode,household_id,unique_id,age,relation_to_household_head
82,Sadie,Katia,Tidwell,2017-10-15,Female,Black,"w 4th st north port, fl",34287,48,1,NaN,NaN
83,Liliana,Addisyn,Marshall,2019-12-03,Female,Black,"w 4th st north port, fl",34287,48,2,NaN,NaN
174,Holly,Emma,Yount,2019-05-17,Female,White,"7944 se 62nd ave unincorporated, fl",32824,88,3,NaN,NaN
306,Emilee,Guadalupe,Haskew,2019-12-30,Female,Latino,"749 mi ridge ests destin, fl",32541,150,4,NaN,NaN
323,Gunner,Liam,Parkinson,2020-03-03,Male,White,"600 n maranantha rd hialeah, fl",33016,157,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
48269,Kaylee,Trinity,Hill,2017-10-20,Female,Black,"98 melanie dr pembroke pines, fl",33026,20380,629,NaN,NaN
48351,Lev,Thomas,Dove,2018-10-18,Male,Black,"671 john muir road spring hill, fl",34610,20422,630,NaN,NaN
48442,Frederick,Cameron,Rodriguez,2019-06-04,Male,Latino,"5765 heards forest dr crestview, fl",32539,20452,631,NaN,NaN
48456,Liam,Emmett,Sardone,2017-01-08,Male,White,"107 brown ave st. petersburg, fl",33704,20458,632,NaN,NaN


# For round 1, match only to under-5-year olds in census

In [9]:
under5 = df_census.age<5
df_census_u5 = df_census.loc[under5]
df_census_u5.shape

(2243, 12)

# Create some blocking rules and make a linker to count comparisons

## Questions:

1. How do I determine a good blocking strategy? 

In [10]:
# Minimal settings needed below
initial_settings = {'link_type': 'link_only'}
linker_test = DuckDBLinker([df_census_u5, df_wic], initial_settings)

In [11]:
blocking_rules_test = [
#     'l.address = r.address',
#     'l.zipcode = r.zipcode',
#     'substr(l.zipcode, 1,3) = substr(r.zipcode, 1,3)',
    'l.first_name = r.first_name and l.last_name = r.last_name',
    'l.sex = r.sex and l.date_of_birth = r.date_of_birth',
#     (
#         'l.sex = r.sex'
#         ' and l.date_of_birth = r.date_of_birth'
#         ' and substr(l.first_name, 1,1) = substr(r.first_name, 1,1)'
#         ' and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)'
#     ),
    'l.address = r.address',
    'l.zipcode = r.zipcode',
    (
        'substr(l.first_name, 1,1) = substr(r.first_name, 1,1)'
        ' and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)'
    ),
#     'substr(l.zipcode, 1,3) = substr(r.zipcode, 1,3)',
]

print('Cartesion product of input:', len(df_census_u5) * len(df_wic), '\n')

for rule in blocking_rules_test:
    count = linker_test.count_num_comparisons_from_blocking_rule(rule)
    print(f"Number of comparisons generated by '{rule}': {count:,.0f}")

linker_test.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules_test)

Cartesion product of input: 1419819 

Number of comparisons generated by 'l.first_name = r.first_name and l.last_name = r.last_name': 545
Number of comparisons generated by 'l.sex = r.sex and l.date_of_birth = r.date_of_birth': 911
Number of comparisons generated by 'l.address = r.address': 821
Number of comparisons generated by 'l.zipcode = r.zipcode': 3,610
Number of comparisons generated by 'substr(l.first_name, 1,1) = substr(r.first_name, 1,1) and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)': 6,074


# Try to estimate how the blocking strategy will scale

In [12]:
90_000/1e6

0.09

In [13]:
90_000 / len(df_census)

1.893580761219466

In [14]:
90_000 / len(df_census)**2 # Ratio should stay about the same when scaling up census

3.984053443622769e-05

In [15]:
# Ratio should stay about the same when scaling up census
90_000 / (len(df_census)*len(df_wic))

0.002991438801294575

In [16]:
len(df_wic)/len(df_census) # Ratio should stay about the same when scaling up census

0.013318184687243578

In [17]:
# Looks like maybe 47 billion comparisons would be generated...
# Probably too many...
(
    (90_000 / len(df_census)**2)
    * (len(df_wic)/len(df_census)) * (300e6)**2
)

47754323609.41513

# 2nd attempt at blocking

Order of rules doesn't matter. All that matters is:

1. Is every true match captured by one of the rules?
2. Is the total number of comparisons computationally feasible?

In [28]:
# Minimal settings needed below
initial_settings = {'link_type': 'link_only'}
linker = DuckDBLinker([df_census_u5, df_wic], initial_settings)

In [19]:
prediction_blocking_rules = [
    (
        'substr(l.first_name, 1,1) = substr(r.first_name, 1,1)'
        ' and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)'
    ),
    'l.date_of_birth = r.date_of_birth',
    'l.zipcode = r.zipcode',
    'l.address = r.address',
    'l.last_name = r.last_name',
    'l.first_name = r.first_name',
#     'l.sex = r.sex and substr(l.zipcode, 1,3) = substr(r.zipcode, 1,3)',
]

print('Cartesion product of input:', len(df_census_u5) * len(df_wic), '\n')

for rule in prediction_blocking_rules:
    count = linker.count_num_comparisons_from_blocking_rule(rule)
    print(f"Number of comparisons generated by '{rule}': {count:,.0f}")

linker.cumulative_num_comparisons_from_blocking_rules_chart(prediction_blocking_rules)

Cartesion product of input: 1419819 

Number of comparisons generated by 'substr(l.first_name, 1,1) = substr(r.first_name, 1,1) and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)': 6,074
Number of comparisons generated by 'l.date_of_birth = r.date_of_birth': 1,302
Number of comparisons generated by 'l.zipcode = r.zipcode': 3,610
Number of comparisons generated by 'l.address = r.address': 821
Number of comparisons generated by 'l.last_name = r.last_name': 1,658
Number of comparisons generated by 'l.first_name = r.first_name': 2,966


# Define comparisons we want to use

In [31]:
comparisons = [
        cl.levenshtein_at_thresholds("first_name"),
        cl.levenshtein_at_thresholds("last_name"),
#         c1.exact_match("last_name"),
        cl.levenshtein_at_thresholds("date_of_birth", 1),
        cl.exact_match("sex"),
        cl.levenshtein_at_thresholds("zipcode"),
    ]

# Create a settings dictionary

In [32]:
settings = {
    "link_type": "link_only",
    "comparisons": comparisons,
    "blocking_rules_to_generate_predictions": prediction_blocking_rules,
    "probability_two_random_records_match": 1/len(df_census_u5),
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

linker.initialise_settings(settings)

In [33]:
%%time
linker.estimate_u_using_random_sampling(target_rows=len(df_wic)*len(df_census_u5))

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - last_name (no m values are trained).
    - date_of_birth (no m values are trained).
    - sex (no m values are trained).
    - zipcode (no m values are trained).


CPU times: user 3.07 s, sys: 0 ns, total: 3.07 s
Wall time: 1.57 s


In [44]:
# training_blocking_rules = [
#     "l.first_name = r.first_name and l.last_name = r.last_name and l.zipcode = r.zipcode",
#     "l.first_name = r.first_name and l.date_of_birth = r.date_of_birth and l.zipcode = r.zipcode",
#     "l.last_name = r.last_name and l.date_of_birth = r.date_of_birth and l.zipcode = r.zipcode",
#     "l.last_name = r.last_name and l.date_of_birth = r.date_of_birth and l.first_name = r.first_name"
# ]

training_blocking_rules = [
    "l.first_name = r.first_name",
    "l.zipcode = r.zipcode",
]

for rule in training_blocking_rules:
    count = linker.count_num_comparisons_from_blocking_rule(rule)
    print(f"Number of comparisons generated by '{rule}': {count:,.0f}")

# linker.cumulative_num_comparisons_from_blocking_rules_chart(training_blocking_rules)

Number of comparisons generated by 'l.first_name = r.first_name': 2,966
Number of comparisons generated by 'l.zipcode = r.zipcode': 3,610


In [25]:
%%time
training_sessions = {}
for rule in training_blocking_rules:
    training_sessions[rule] = linker.estimate_parameters_using_expectation_maximisation(rule)
    


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name

Parameter estimates will be made for the following comparison(s):
    - last_name
    - date_of_birth
    - sex
    - zipcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name

Iteration 1: Largest change in params was 0.0441 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.000417 in the m_probability of last_name, level `Exact match`
Iteration 3: Largest change in params was 2.24e-05 in the m_probability of last_name, level `Exact match`

EM converged after 3 iterations

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.zipcode = r.zipcode

Parameter estimates will be made for the follow

CPU times: user 321 ms, sys: 27 ms, total: 348 ms
Wall time: 329 ms


In [26]:
linker.m_u_parameters_chart()

In [27]:
# "comparisons": [
#         levenshtein("first_name", 2, term_frequency_adjustments=True),
#         levenshtein("last_name", 2, term_frequency_adjustments=True),
#         levenshtein("dob", 2, term_frequency_adjustments=True),
#         exact_match("sex"),
#         levenshtein("zip", 2, term_frequency_adjustments=True),
#         exact_match("ssn"),
#     ],

In [34]:
linker._settings_dict

{'link_type': 'link_only',
 'comparisons': [<Comparison Exact match vs. levenshtein at thresholds 1, 2 vs. anything else with 4 levels at 0x7ff8ac105280>,
  <Comparison Exact match vs. levenshtein at thresholds 1, 2 vs. anything else with 4 levels at 0x7ff8ac105cd0>,
  <Comparison Exact match vs. levenshtein at threshold 1 vs. anything else with 3 levels at 0x7ff8ac105e80>,
  <Comparison Exact match vs. anything else with 2 levels at 0x7ff8ac105400>,
  <Comparison Exact match vs. levenshtein at thresholds 1, 2 vs. anything else with 4 levels at 0x7ff8ac105dc0>],
 'blocking_rules_to_generate_predictions': ['substr(l.first_name, 1,1) = substr(r.first_name, 1,1) and substr(l.last_name, 1,1) = substr(r.last_name, 1,1)',
  'l.date_of_birth = r.date_of_birth',
  'l.zipcode = r.zipcode',
  'l.address = r.address',
  'l.last_name = r.last_name',
  'l.first_name = r.first_name'],
 'probability_two_random_records_match': 0.00044583147570218456,
 'retain_matching_columns': True,
 'retain_intermed

In [38]:
1/len(df_census_u5)

0.00044583147570218456

In [37]:
print(linker._settings_obj.human_readable_description)

SUMMARY OF LINKING MODEL
------------------------
The similarity of pairwise record comparison in your model will be assessed as follows:

Comparison of "first_name"
Description: 'Exact match vs. levenshtein at thresholds 1, 2 vs. anything else'
Comparison levels:
    - 'Null' with SQL rule: "first_name_l" IS NULL OR "first_name_r" IS NULL
    - 'Exact match' with SQL rule: "first_name_l" = "first_name_r"
    - 'levenshtein <= 1' with SQL rule: levenshtein("first_name_l", "first_name_r") <= 1
    - 'levenshtein <= 2' with SQL rule: levenshtein("first_name_l", "first_name_r") <= 2
    - 'All other comparisons' with SQL rule: ELSE

Comparison of "last_name"
Description: 'Exact match vs. levenshtein at thresholds 1, 2 vs. anything else'
Comparison levels:
    - 'Null' with SQL rule: "last_name_l" IS NULL OR "last_name_r" IS NULL
    - 'Exact match' with SQL rule: "last_name_l" = "last_name_r"
    - 'levenshtein <= 1' with SQL rule: levenshtein("last_name_l", "last_name_r") <= 1
    - 'lev

In [43]:
print(linker._settings_obj_.salting_required)

False
